In [39]:
# Data
# https://www.kaggle.com/siddarthareddyt/cats-and-dogs/data

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

%matplotlib inline

In [42]:
train_path='/Users/administrator/Research/trainings/learnings/data/cats-and-dogs/train'
valid_path='/Users/administrator/Research/trainings/learnings/data/cats-and-dogs/valid'
test_path='/Users/administrator/Research/trainings/learnings/data/cats-and-dogs/test'

#train_path='/home/sandeep/datasets/dataset/dataset/training_set'
#valid_path='/home/sandeep/datasets/dataset/dataset/validation_set'

In [77]:
train_batches=ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),classes=['dogs','cats'],batch_size=3)
valid_batches=ImageDataGenerator().flow_from_directory(valid_path,target_size=(224,224),classes=['dogs','cats'],batch_size=3)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [58]:

def plots(ims,figsize=(12,6),rows=2,interp=False,titles=None):
    if type(ims[0]) is np.ndarray:
        ims=np.array(ims).astype(np.uint8)
        if(ims.shape[-1]!=3):
            ims=ims.transpose((0,2,3,1))
    f=plt.figure(figsize=(100,100))
    cols=len(ims)//rows if len(ims)%2==0 else len(ims)//rows+1
    for i in range(len(ims)):
        sp=f.add_subplot(rows,cols,i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i],fontsize=16)
        plt.imshow(ims[i],interpolation=none if interp else 'none')

In [59]:
imgs,labels=next(train_batches)

In [12]:
#plots(imgs,titles=labels)
# Dogs: [1,0]
# Cats[0,1]

In [9]:
# Shape of the batch of images
#for img in imgs:
#    print(img.shape)

In [6]:
model = Sequential(
[
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    Flatten(),
    Dense(2,activation='softmax')
]
)

In [7]:
model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.fit_generator(train_batches,steps_per_epoch=1,validation_data=valid_batches,validation_steps=50,epochs=5,verbose=1)

Epoch 1/5
1/1 [==============================] - 40s 40s/step - loss: 7.1771 - acc: 0.4500 - val_loss: 8.0107 - val_acc: 0.5030
Epoch 2/5
1/1 [==============================] - 3s 3s/step - loss: 8.0590 - acc: 0.5000 - val_loss: 7.9785 - val_acc: 0.5050
Epoch 3/5
1/1 [==============================] - 3s 3s/step - loss: 6.4472 - acc: 0.6000 - val_loss: 7.9623 - val_acc: 0.5060
Epoch 4/5
1/1 [==============================] - 3s 3s/step - loss: 7.2531 - acc: 0.5500 - val_loss: 8.4942 - val_acc: 0.4730
Epoch 5/5
1/1 [==============================] - 3s 3s/step - loss: 6.4472 - acc: 0.6000 - val_loss: 7.8495 - val_acc: 0.5130


# Fine tuning the existing model : vgg16

In [46]:
vgg16_model=keras.applications.vgg16.VGG16()

In [47]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [48]:
type(vgg16_model)

keras.engine.training.Model

In [49]:
model=Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [51]:
model.layers.pop()

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [53]:
for layer in model.layers:
    layer.trainable = False

In [54]:
model.add(Dense(2,activation='softmax'))

In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [78]:
model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [80]:
model.fit_generator(train_batches,steps_per_epoch=10,validation_data=valid_batches,validation_steps=5,epochs=1,verbose=1)

Epoch 1/1
10/10 [==============================] - 23s 2s/step - loss: 0.5900 - acc: 0.9333 - val_loss: 0.6020 - val_acc: 0.8667
